In [1]:
#####################################################################################################################
## LDA-COHERENCE-ZILLA  
## Fuentes : https://markroxor.github.io/gensim/static/notebooks/topic_coherence_tutorial.html
##             https://medium.com/@lettier/how-does-lda-work-ill-explain-using-emoji-108abf40fa7d
##             http://phusewiki.org/wiki/images/c/c9/Weizhong_Presentation_CDER_Nov_9th.pdf
##             The Art and Science of Analyzing Sotfware Data, Christian Bird et al
##             https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html
##             https://www.quora.com/What-should-I-clean-from-text-before-LDA
##             https://www.machinelearningplus.com/nlp/lemmatization-examples-python/
##             https://github.com/derekgreene/topic-model-tutorial/blob/master/3%20-%20Parameter%20Selection%20for%20NMF.ipynb
##             https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/
##             https://radimrehurek.com/gensim/models/ldamodel.html
##             https://radimrehurek.com/gensim/models/coherencemodel.html
##             https://towardsdatascience.com/light-on-math-machine-learning-intuitive-guide-to-latent-dirichlet-allocation-437c81220158
##             https://towardsdatascience.com/evaluate-topic-model-in-python-latent-dirichlet-allocation-lda-7d57484bb5d0
##             https://towardsdatascience.com/latent-dirichlet-allocation-lda-9d1cd064ffa2
##             https://stats.stackexchange.com/questions/59684/what-are-typical-values-to-use-for-alpha-and-beta-in-latent-dirichlet-allocation
##             https://towardsdatascience.com/end-to-end-topic-modeling-in-python-latent-dirichlet-allocation-lda-35ce4ed6b3e0
##             https://medium.com/@yanlinc/how-to-build-a-lda-topic-model-using-from-text-601cdcbfd3a6
#############################################by#######JMG############2020################################################
## Estructura del Código:
## - Importación de librerias
## - Funciones
## - Carga de archivo CSV
## - Limpieza
## - Segmentación de noticias por semana desde el 13-03 al 14-04
## - Visualización - Semana 1 
## - Visualización - Semana 2
## - Visualización - Semana 3
## - Visualización - Semana 4



import pandas as pd
import numpy as np
import re
import gensim.corpora as corpora
import gensim
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess
import spacy
import nltk
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
import pyLDAvis.gensim
import pickle 
import pyLDAvis
from pprint import pprint
from  sklearn.feature_extraction.text import CountVectorizer







In [2]:
## - Funciones

def remove_stopwords(texts):
    nltk.download('stopwords')
    stop_words = stopwords.words('spanish')
    newStopWords = ['coronavirus','chile','casos','detalles','agrego','embargo','forma','semanas','fecha','francia','espana','cada','agregó','parte','caso','dias','respecto','momento','covid','ascom','tikitakas','personas','tipo','tenía','jorge','habrá','habra','medios','perú','casa','cuales',
                'marzo','abril','lunes','semana','despues','hora','también','tambien','martes','radio','lugar','lugares','anuncio','anunciar',
                'miércoles','jueves','viernes','sábado','domingo','enero','comentarios','reservamos','febrero','italia','según','segun','horas','aquí','virus','veces','radio','eliodoro', 'yanez', 'providencia',
                 'reservamos','derecho','fabiola','shtml','pictwittercom','arrow','biobiochile','https_www','bloquear','april','usuarios','agresivos','commons','noticias','indique','contrario','eliminar','comentarios','debate','espacio','wordpress','desarrollado','detallo','estime','conveniente','manteniendo',]
    press_vect = CountVectorizer()
    press_vect.fit(df_corpus.contenido)

    # Transformamos el corpus en una representación matricial
    X_press = press_vect.transform(df_corpus.contenido)

    # Creamos una tabla para visualizar la representación matricial
    X_df=pd.DataFrame(X_press.toarray(), columns=press_vect.get_feature_names())
    df_s=X_df.sum(numeric_only=True)
    df_2=df_s.sort_values(ascending=False)
    masfrec = (df_2[:20:])
    list_mas= masfrec.index.tolist()
    menosfrec = (df_2.iloc[-20:])
    list_menos= masfrec.index.tolist()
    
    
    stop_words.extend(newStopWords)    
    
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]
def make_bigrams(texts,bigram_mod):
    return [bigram_mod[doc] for doc in texts]


def to_array(texts):
    arreglo_v=[]
    for docv in range(len(texts)):
        for wordv in data_lemmatized[docv].split():
            if wordv is None:
                pass
            else:
                arreglo_v.append(wordv)
    return arreglo_v
def lemma_sustantivos(texts):
    nlp1 = spacy.load("es_core_news_sm")
    text_out = []
    for sent in texts:
        doc = nlp1(" ".join(sent)) 
        text_out.append(" ".join([token.text for token in doc.noun_chunks])) 
    return text_out

def coherence_zilla(corpus,dictionary,dataset,texts,tmin,tmax):
    
    min_topics = tmin
    max_topics = tmax
    step_size = 1
    topics_range = range(min_topics, max_topics, step_size)
    alpha = list(np.arange(0.01, 1, 0.3))
    beta  = list(np.arange(0.01,0.1, 0.03))
    num_of_docs = len(corpus)
    max_coherence = 0
    best_model = dict();

    corpus_sets = gensim.utils.ClippedCorpus(corpus, num_of_docs*0.8),
    for k in topics_range:
        for a in alpha:
            for b in beta:
                lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=k, 
                                           random_state=100,
                                           chunksize=50,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
                              
                coherence_model_lda = CoherenceModel(model=lda_model, corpus=corpus,texts=texts, coherence='c_v')
                cv = coherence_model_lda.get_coherence()
                if max_coherence < cv:
                    max_coherence=cv
                    best_alpha=a
                    best_beta=b
                    best_topics=k
                  
                else:
                    pass
         
                    
      
    best_model['coherence'] = max_coherence
    best_model['alpha']   = best_alpha
    best_model['beta'] = best_beta
    best_model['topics'] =  best_topics
    
    return best_model

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
        
def pre_proceso(df_corpus):
    datos= df_corpus.values.tolist()
    datos_palabras= list(sent_to_words(datos))
    
    bigram = gensim.models.Phrases(datos_palabras,min_count=5) 
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    data_words_nostops = remove_stopwords(datos_palabras)
    data_words_bigrams = make_bigrams(data_words_nostops,bigram_mod)
    data_lemmatized =  lemma_sustantivos(data_words_bigrams)
    return data_lemmatized
            

In [3]:
## - Carga de archivo CSV

df_file = pd.read_csv('cleaning_contenido3.csv')

In [6]:
## - Limpieza

df_corpus= df_file
df_corpus=df_corpus[1:]
df_corpus=df_corpus.dropna()
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('\s+', ' ', x))
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('[_,";¿*&()=:+#\|@.!?]', ' ', x))
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: x.lower())
palabras_cortas = re.compile(r'\W*\b\w{1,3}\b')
df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: palabras_cortas.sub('', x))

df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: re.sub('\s+', ' ', x))
df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: re.sub('[_,";¿*&()=:+#\|@.!?]', ' ', x))
df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: x.lower())
palabras_cortas = re.compile(r'\W*\b\w{1,3}\b')
df_corpus['titulos'] = df_corpus['titulos'].map(lambda x: palabras_cortas.sub('', x))
df_corpus['contenido'] = df_corpus['titulos']+" "+df_corpus['contenido']





<>:6: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \|
<>:12: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \|
<>:6: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \|
<>:12: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \|
<>:6: DeprecationWarning: invalid escape sequence \s
<>:7: DeprecationWarning: invalid escape sequence \|
<>:12: DeprecationWarning: invalid escape sequence \s
<>:13: DeprecationWarning: invalid escape sequence \|
<ipython-input-6-fc99840d8756>:6: DeprecationWarning: invalid escape sequence \s
  df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('\s+', ' ', x))
<ipython-input-6-fc99840d8756>:7: DeprecationWarning: invalid escape sequence \|
  df_corpus['contenido'] = df_corpus['contenido'].map(lambda x: re.sub('[_,";¿*&()=:+#\|@.!?]', ' ', x))
<ipy

NameError: name 'word_tokenize' is not defined

In [5]:
## - Segmentación de noticias por semana desde el 13-03 al 14-04

df_semana_1 = df_corpus[(df_corpus['fecha publicacion'] >= '2020-03-01') & (df_corpus['fecha publicacion'] <= '2020-03-23')]
df_semana_2 = df_corpus[(df_corpus['fecha publicacion'] > '2020-03-23') & (df_corpus['fecha publicacion'] <= '2020-03-30')]
df_semana_3 = df_corpus[(df_corpus['fecha publicacion'] > '2020-03-30') & (df_corpus['fecha publicacion'] <= '2020-04-07')]
df_semana_4 = df_corpus[(df_corpus['fecha publicacion'] > '2020-04-07') & (df_corpus['fecha publicacion'] <= '2020-04-14')]


In [6]:
############# Visualización - Semana 1 ##########
s_lemma_1 = pre_proceso(df_semana_1)
dataset_1 = [d.split() for d in s_lemma_1]
id2word_1 = corpora.Dictionary(dataset_1)
texts_1 = dataset_1
corpus_1 = [id2word_1.doc2bow(text) for text in texts_1]

s1_model = coherence_zilla(corpus_1,id2word_1,dataset_1,texts_1,15,16)
lda_model_s1 = gensim.models.LdaMulticore(corpus=corpus_1,
                                       id2word=id2word_1,
                                       num_topics=s1_model.get('topics'), 
                                       alpha=s1_model.get('alpha'), 
                                       eta=s1_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    
pyLDAvis.enable_notebook()
LDAvis_prepared_1 = pyLDAvis.gensim.prepare(lda_model_s1, corpus_1,id2word_1)
LDAvis_prepared_1



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmerc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated,

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.017771  0.110427       1        1  47.578548
10     0.217146  0.002969       2        1  35.754101
7     -0.030143  0.076289       3        1   3.966825
6      0.001523 -0.073340       4        1   2.971935
5     -0.039833 -0.058497       5        1   2.887390
9     -0.033556 -0.029743       6        1   2.055752
3     -0.022669 -0.004232       7        1   1.871968
12    -0.000222  0.000479       8        1   0.785840
8     -0.022240 -0.006861       9        1   0.758673
4     -0.008302 -0.002537      10        1   0.228163
13    -0.008455 -0.002881      11        1   0.228163
14    -0.009103 -0.002463      12        1   0.228162
11    -0.008588 -0.003055      13        1   0.228162
2     -0.009105 -0.003126      14        1   0.228162
0     -0.008685 -0.003429      15        1   0.228162, topic_info=           Term       Freq      Total Category  logprob  loglift
38        salud  47.000000  47.000000  Default  30.0000  30.0000
11      medidas  48.000000  48.000000  Default  29.0000  29.0000
18      persona  10.000000  10.000000  Default  28.0000  28.0000
19       region  15.000000  15.000000  Default  27.0000  27.0000
9       informo  13.000000  13.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
374    pandemia   0.006227  22.276731  Topic15  -7.0934  -2.0995
212  enfermedad   0.006195  15.811373  Topic15  -7.0986  -1.7619
397        plan   0.006191  15.917475  Topic15  -7.0991  -1.7691
358        nota   0.006188  11.369489  Topic15  -7.0996  -1.4331
149     comunas   0.006182  11.372568  Topic15  -7.1007  -1.4345

[916 rows x 6 columns], token_table=      Topic      Freq             Term
term                                  
55        1  0.882290         acciones
709       2  0.941509         acuerdos
533       1  0.561985     adolescentes
60        1  0.551876  adultos_mayores
60        2  0.344923  adultos_mayores
...     ...       ...              ...
524       1  0.814336         vigencia
524       5  0.203584         vigencia
1010      2  0.998599  wwwipsenlineacl
1243      3  1.037837          yoihiay
1011      2  1.013701             zona

[428 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 11, 8, 7, 6, 10, 4, 13, 9, 5, 14, 15, 12, 3, 1])

In [14]:

# 10 topicos -semana 1
pprint(lda_model_s1.print_topics())
print(s1_model)


[(0,
  '0.001*"medidas" + 0.001*"cuarentena" + 0.001*"salud" + 0.001*"region" + '
  '0.001*"pacientes" + 0.001*"contagios" + 0.001*"gobierno" + 0.001*"pais" + '
  '0.001*"crisis" + 0.001*"pinera"'),
 (1,
  '0.020*"salud" + 0.016*"medidas" + 0.013*"cuarentena" + 0.011*"pais" + '
  '0.009*"pandemia" + 0.009*"pinera" + 0.008*"gobierno" + 0.007*"presidente" + '
  '0.007*"general_ejercito" + 0.006*"contagio"'),
 (2,
  '0.001*"medidas" + 0.001*"salud" + 0.001*"pais" + 0.001*"cuarentena" + '
  '0.001*"region" + 0.001*"gobierno" + 0.001*"pacientes" + 0.001*"mundo" + '
  '0.001*"crisis" + 0.001*"contagio"'),
 (3,
  '0.038*"deportistas" + 0.022*"dinero" + 0.022*"perez" + 0.017*"rendimiento" '
  '+ 0.011*"salud" + 0.011*"emergencia" + 0.011*"cecilia" + 0.011*"eventos" + '
  '0.011*"deportes" + 0.006*"julio"'),
 (4,
  '0.001*"salud" + 0.001*"medidas" + 0.001*"pais" + 0.001*"cuarentena" + '
  '0.001*"pinera" + 0.001*"contagio" + 0.001*"region" + 0.001*"gobierno" + '
  '0.001*"pandemia" + 0.001*"pla

In [ ]:
############# Visualización - Semana 2 ##########
s_lemma_2 = pre_proceso(df_semana_2)
dataset_2 = [d.split() for d in s_lemma_2]
id2word_2 = corpora.Dictionary(dataset_2)
texts_2 = dataset_2
corpus_2 = [id2word_2.doc2bow(text) for text in texts_2]

s2_model = coherence_zilla(corpus_2,id2word_2,dataset_2,texts_2,10,17)
lda_model_s2 = gensim.models.LdaMulticore(corpus=corpus_2,
                                       id2word=id2word_2,
                                       num_topics=s2_model.get('topics'), 
                                       alpha=s2_model.get('alpha'), 
                                       eta=s2_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    

LDAvis_prepared_2 = pyLDAvis.gensim.prepare(lda_model_s2, corpus_2,id2word_2)
LDAvis_prepared_2


In [ ]:
# 10 topicos -semana 2
pprint(lda_model_s2.print_topics())


In [12]:
############# Visualización - Semana 3
s_lemma_3 = pre_proceso(df_semana_3)
dataset_3 = [d.split() for d in s_lemma_3]
id2word_3 = corpora.Dictionary(dataset_3)
texts_3 = dataset_3
corpus_3 = [id2word_3.doc2bow(text) for text in texts_3]

s3_model = coherence_zilla(corpus_3,id2word_3,dataset_3,texts_3,8,11)
lda_model_s3 = gensim.models.LdaMulticore(corpus=corpus_3,
                                       id2word=id2word_3,
                                       num_topics=s3_model.get('topics'), 
                                       alpha=s3_model.get('alpha'), 
                                       eta=s3_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    

LDAvis_prepared_3 = pyLDAvis.gensim.prepare(lda_model_s3, corpus_3,id2word_3)
LDAvis_prepared_3


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmerc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated,

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.083020  0.041917       1        1  50.626999
5     -0.146575 -0.005684       2        1  34.652054
0     -0.138090 -0.222350       3        1   5.008293
8      0.225228 -0.234463       4        1   3.621889
7      0.182942  0.079292       5        1   3.137577
6      0.142074  0.137635       6        1   1.873685
9     -0.042501  0.047752       7        1   0.269875
4     -0.046507  0.053632       8        1   0.269875
2     -0.047820  0.052273       9        1   0.269875
1     -0.045732  0.049997      10        1   0.269875, topic_info=           Term       Freq      Total Category  logprob  loglift
361       salud  40.000000  40.000000  Default  30.0000  30.0000
83     pandemia  42.000000  42.000000  Default  29.0000  29.0000
189  cuarentena  39.000000  39.000000  Default  28.0000  28.0000
65      medidas  54.000000  54.000000  Default  27.0000  27.0000
234    examenes  13.000000  13.000000  Default  26.0000  26.0000
..          ...        ...        ...      ...      ...      ...
118      tiempo   0.012430  17.553646  Topic10  -6.7592  -1.3379
69     millones   0.013048  29.338335  Topic10  -6.7106  -1.8030
357      riesgo   0.011677  12.746941  Topic10  -6.8217  -1.0804
258     informo   0.011697  14.672282  Topic10  -6.8200  -1.2194
364    santiago   0.011635  17.507925  Topic10  -6.8253  -1.4014

[703 rows x 6 columns], token_table=      Topic      Freq                 Term
term                                      
3         2  1.009441  activos_financieros
1396      3  1.006764               actriz
750       1  0.346353           actualidad
750       3  0.692707           actualidad
123       1  0.201738        actualizacion
...     ...       ...                  ...
730       1  0.941389             virologo
917       6  1.030026               vuelta
1228      4  0.977249       vulnerabilidad
1442      3  1.006787                 york
1389      5  0.989477                  zej

[638 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 6, 1, 9, 8, 7, 10, 5, 3, 2])

In [15]:
# 10 topicos -semana 3
pprint(lda_model_s3.print_topics())
print(s3_model)

[(0,
  '0.044*"salud" + 0.020*"infectados_nuevos" + 0.018*"camas" + '
  '0.018*"tratadas" + 0.018*"contagios" + 0.017*"pais" + 0.015*"hospitales" + '
  '0.014*"constitucion" + 0.014*"supero" + 0.013*"ciudadanos"'),
 (1,
  '0.003*"cuarentena" + 0.002*"medidas" + 0.002*"gobierno" + 0.002*"pandemia" '
  '+ 0.002*"salud" + 0.002*"region" + 0.002*"crisis" + 0.002*"mundo" + '
  '0.002*"ministerio_salud" + 0.002*"jaime_manalich"'),
 (2,
  '0.003*"medidas" + 0.002*"pais" + 0.002*"cuarentena" + 0.002*"salud" + '
  '0.002*"pandemia" + 0.002*"gobierno" + 0.001*"muertes" + 0.001*"region" + '
  '0.001*"mismo" + 0.001*"contagios"'),
 (3,
  '0.013*"crisis" + 0.012*"salud" + 0.012*"medidas" + 0.011*"cuarentena" + '
  '0.010*"aire" + 0.010*"gobierno" + 0.009*"pandemia" + 0.007*"mundo" + '
  '0.007*"manalich" + 0.006*"propagacion"'),
 (4,
  '0.002*"millones" + 0.002*"cuarentena" + 0.002*"medidas" + 0.002*"gobierno" '
  '+ 0.002*"salud" + 0.001*"pandemia" + 0.001*"crisis" + 0.001*"region" + '
  '0.001*"b

In [16]:
############# Visualización - Semana 4
s_lemma_4 = pre_proceso(df_semana_4)
dataset_4 = [d.split() for d in s_lemma_4]
id2word_4 = corpora.Dictionary(dataset_4)
texts_4 = dataset_4
corpus_4 = [id2word_4.doc2bow(text) for text in texts_4]

s4_model = coherence_zilla(corpus_4,id2word_4,dataset_4,texts_4,13,14)
lda_model_s4 = gensim.models.LdaMulticore(corpus=corpus_4,
                                       id2word=id2word_4,
                                       num_topics=s4_model.get('topics'), 
                                       alpha=s4_model.get('alpha'), 
                                       eta=s4_model.get('beta'), 
                                       random_state=100,
                                       chunksize=50,
                                       passes=10,
                                       per_word_topics=True)
    



[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jmerc\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated, Use raw=False instead.
  return _unpackb(packed, **kwargs)
C:\ProgramData\Anaconda3\envs\simple\lib\site-packages\msgpack_numpy.py:184: DeprecationWarning: encoding is deprecated,

In [17]:

LDAvis_prepared_4 = pyLDAvis.gensim.prepare(lda_model_s4, corpus_4,id2word_4)
LDAvis_prepared_4

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.161049  0.004876       1        1  30.815104
12     0.136331 -0.015592       2        1  23.612062
9      0.003144  0.139152       3        1  11.059119
0      0.061631  0.026732       4        1  10.207649
8      0.006160 -0.170455       5        1   7.644757
10    -0.070444 -0.044113       6        1   5.051864
5     -0.109835  0.018327       7        1   4.454633
7     -0.017715  0.037322       8        1   2.810156
6     -0.033310  0.005457       9        1   1.509609
2     -0.037576 -0.001286      10        1   1.112635
4     -0.029405 -0.001147      11        1   0.801094
11    -0.035573  0.003076      12        1   0.468085
1     -0.034457 -0.002349      13        1   0.453226, topic_info=               Term       Freq      Total Category  logprob  loglift
203      cuarentena  50.000000  50.000000  Default  30.0000  30.0000
289           mundo  34.000000  34.000000  Default  29.0000  29.0000
18          muertos  14.000000  14.000000  Default  28.0000  28.0000
274         medidas  51.000000  51.000000  Default  27.0000  27.0000
234          examen  17.000000  17.000000  Default  26.0000  26.0000
..              ...        ...        ...      ...      ...      ...
289           mundo   0.015600  34.336689  Topic13  -7.4766  -2.3002
16   ministro_salud   0.015318  17.769901  Topic13  -7.4948  -1.6597
308         persona   0.015315  23.439573  Topic13  -7.4950  -1.9368
310          pinera   0.015226  15.315131  Topic13  -7.5008  -1.5171
123        santiago   0.015252  22.717453  Topic13  -7.4991  -1.9097

[740 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
911       8  0.863433      abril
762       2  0.178842   acciones
762       4  0.357685   acciones
762       5  0.357685   acciones
1154      3  0.701072  acontecio
...     ...       ...        ...
28        1  0.404241       zona
28        2  0.134747       zona
28        3  0.067374       zona
28        4  0.202121       zona
28        5  0.134747       zona

[985 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 13, 10, 1, 9, 11, 6, 8, 7, 3, 5, 12, 2])

In [18]:
# 10 topicos -semana 4
pprint(lda_model_s4.print_topics())

[(0,
  '0.018*"cuarentena" + 0.012*"region" + 0.011*"brote" + 0.010*"pandemia" + '
  '0.010*"brotes" + 0.009*"crisis" + 0.008*"enfermedad" + 0.008*"pacientes" + '
  '0.008*"autoridades" + 0.007*"formas"'),
 (1,
  '0.005*"mediodia" + 0.005*"atacama" + 0.001*"region" + 0.001*"pandemia" + '
  '0.001*"contagiados" + 0.001*"pais" + 0.001*"crisis" + '
  '0.001*"ministerio_salud" + 0.001*"cantidad" + 0.001*"enfermedad"'),
 (2,
  '0.016*"ciudad" + 0.013*"carabineros" + 0.009*"materia" + '
  '0.008*"funcionarios" + 0.004*"hombres" + 0.004*"estamento" + 0.004*"mando" '
  '+ 0.004*"saavedra" + 0.004*"palacio" + 0.004*"revelo"'),
 (3,
  '0.021*"region" + 0.017*"salud" + 0.015*"medidas" + 0.015*"cuarentena" + '
  '0.011*"muertes" + 0.010*"gobierno" + 0.010*"ministerio_salud" + '
  '0.008*"contagios" + 0.007*"pais" + 0.007*"informo"'),
 (4,
  '0.018*"empleados" + 0.014*"empleadores" + 0.009*"autoridades_sanitarias" + '
  '0.009*"fuerza" + 0.007*"cuarentena" + 0.005*"puestos" + 0.005*"realizacion" '


In [19]:
print(s4_model)

{'coherence': 0.43338591745400235, 'alpha': 0.9099999999999999, 'beta': 0.09999999999999999, 'topics': 13}
